In [6]:
from utils.tentmapdataset import TentDataset

# print an example instance of the dataset
n = 4
length = 22
train_dataset = TentDataset("train", length=length, n_iterations=n)
test_dataset = TentDataset("test", length=length, n_iterations=n)

x, y = train_dataset[0]

print("x:", x)
print("y:", y)

x, y = test_dataset[0]

print("x:", x)
print("y:", y)

x: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
y: tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])
x: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
y: tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  0,  0,  0,  0,  0,  0])


In [7]:
# create a GPT instance
from mingpt.model import GPT
from mingpt.utils import CfgNode as CN

model_config = CN(
    n_layer=3,
    n_head=3,
    n_embd=2**4 * 3,
    model_type=None,
    vocab_size=train_dataset.get_vocab_size(),
    block_size=train_dataset.get_block_size(),
    embd_pdrop=0.1,
    attn_pdrop=0.1,
    resid_pdrop=0.1,
)

model = GPT(model_config)

print(f"Number of training samples: {len(train_dataset):.3e}")
print(f"Number of test samples: {len(test_dataset):.3e}")

number of parameters: 8.707e+04
Number of training samples: 3.146e+06
Number of test samples: 1.049e+06


In [8]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 3e-4
train_config.max_iters = 3000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)
# %%


def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(
            f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}"
        )


trainer.set_callback("on_batch_end", batch_end_callback)


running on device cpu


In [9]:
trainer.run()

iter_dt 0.00ms; iter 0: train loss 0.69479
iter_dt 70.07ms; iter 100: train loss 0.57613
iter_dt 82.93ms; iter 200: train loss 0.56975
iter_dt 109.79ms; iter 300: train loss 0.54576
iter_dt 99.64ms; iter 400: train loss 0.52182
iter_dt 131.96ms; iter 500: train loss 0.51991
iter_dt 90.99ms; iter 600: train loss 0.51178
iter_dt 90.18ms; iter 700: train loss 0.48854
iter_dt 95.15ms; iter 800: train loss 0.39811
iter_dt 90.98ms; iter 900: train loss 0.30842
iter_dt 162.11ms; iter 1000: train loss 0.18512
iter_dt 97.84ms; iter 1100: train loss 0.04567
iter_dt 81.52ms; iter 1200: train loss 0.03101
iter_dt 77.57ms; iter 1300: train loss 0.02026
iter_dt 80.16ms; iter 1400: train loss 0.00610
iter_dt 97.45ms; iter 1500: train loss 0.02016
iter_dt 88.26ms; iter 1600: train loss 0.01014
iter_dt 102.90ms; iter 1700: train loss 0.01223
iter_dt 109.82ms; iter 1800: train loss 0.01270
iter_dt 109.09ms; iter 1900: train loss 0.01069
iter_dt 119.81ms; iter 2000: train loss 0.00816
iter_dt 77.94ms; it

In [10]:
model.eval()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(2, 48)
    (wpe): Embedding(43, 48)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-2): 3 x Block(
        (ln_1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=48, out_features=144, bias=True)
          (c_proj): Linear(in_features=48, out_features=48, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=48, out_features=192, bias=True)
          (c_proj): Linear(in_features=192, out_features=48, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=48, out_f

In [11]:
import torch
from torch.utils.data.dataloader import DataLoader


def eval_split(trainer, split, max_batches):
    dataset = {"train": train_dataset, "test": test_dataset}[split]
    n = train_dataset.length
    results = []
    mistakes = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        # isolate the input pattern alone
        inp = x[:, :n]
        sol = y[:, -n:]
        # let the model sample the rest of the sequence
        cat = model.generate(
            inp, n, do_sample=False
        )  # using greedy argmax, not sampling
        sol_candidate = cat[:, n:]  # isolate the filled in sequence
        # compare the predicted sequence to the true sequence
        correct = (sol == sol_candidate).all(1).cpu()
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            if (
                not correct[i] and "".join(map(str, inp[i].tolist())) not in mistakes
            ):  # and mistakes_printed_already < 3  # only print up to 5 mistakes to get a sense
                mistakes_printed_already += 1
                mistakes.append("".join(map(str, inp[i].tolist())))
                print(
                    "GPT claims that %s -> %s but g.t. is %s"
                    % (inp[i].tolist(), sol_candidate[i].tolist(), sol[i].tolist())
                )
        if max_batches is not None and b + 1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print(
        "%s final score: %d/%d = %.2f%% correct"
        % (split, rt.sum(), len(results), 100 * rt.mean())
    )
    return rt.sum()

In [ ]:
# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    # train_score = eval_split(trainer, "train", max_batches=None)  # 50
    test_score = eval_split(trainer, "test", max_batches=None)  # 50


KeyboardInterrupt: 

In [ ]:
# let's run a random given sequence through the model as well
n = train_dataset.length  # naugy direct access shrug
inp, sol = train_dataset[3]
inp = inp[:n]
sol = sol[-n:]

inp = inp.unsqueeze(0).to(trainer.device)
sol = sol.unsqueeze(0).to(trainer.device)

assert inp[0].nelement() == n
with torch.no_grad():
    cat = model.generate(inp, n, do_sample=False)

sol_candidate = cat[:, n:]
print("input sequence  :", inp.tolist())
print("output:         ", sol.tolist())
print("predicted:      ", sol_candidate.tolist())
# print('gt sort         :', sol.tolist())
print("matches         :", bool((sol == sol_candidate).all()))

input sequence  : [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
output:          [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
predicted:       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
matches         : True
